In [149]:
! python -m pip install ahpy
! pip install PrettyTable
! pip install pandas
! pip install jinja2

In [150]:
import numpy as np
import ahpy
import prettytable

In [151]:
cov_users = 'covered users'
impl_speed = 'implementation speed'
conv_users = 'convenience for users'
impl_cost = 'implementation cost'

In [152]:
android = 'Publish android app'
web_version = 'Improve web version for smartphones'
extended_old_sup = 'Extended support for old versions'
limited_old_sup = 'Limited support for old versions'

In [153]:
criteria_comparisons = {
    (cov_users, impl_speed): 1,
    (cov_users, conv_users): 5,
    (cov_users, impl_cost): 5,
    (impl_speed, conv_users): 3,
    (impl_speed, impl_cost): 7,
    (conv_users, impl_cost): 1,
}
print(criteria_comparisons)

{('covered users', 'implementation speed'): 1, ('covered users', 'convenience for users'): 5, ('covered users', 'implementation cost'): 5, ('implementation speed', 'convenience for users'): 3, ('implementation speed', 'implementation cost'): 7, ('convenience for users', 'implementation cost'): 1}


In [154]:
# for criteria of covered users
crit_cov_users = {
    (android, web_version): 1/5,
    (android, extended_old_sup) : 1/3,
    (android, limited_old_sup) : 1/3,
    (web_version, extended_old_sup) : 5,
    (web_version, limited_old_sup) : 5,
    (extended_old_sup, limited_old_sup): 1,
}

In [155]:
# for criteria of implementation speed
crit_impl_speed = {
    (android, web_version): 7, 
    (android, extended_old_sup) : 5,
    (android, limited_old_sup) : 3,
    (web_version, extended_old_sup) : 1/3,
    (web_version, limited_old_sup) : 1/3,
    (extended_old_sup, limited_old_sup) : 1/3
}

In [156]:
# for criteria of convenience for users
crit_conv_users = { 
    (android, web_version): 7,
    (android, extended_old_sup) : 5,
    (android, limited_old_sup) : 5,
    (web_version, extended_old_sup): 1/3,
    (web_version, limited_old_sup) : 1/3,
    (extended_old_sup, limited_old_sup) : 2,
}

In [157]:
# for criteria of implementation cost
crit_impl_cost = {
    (android, web_version): 7,
    (android, extended_old_sup): 9,
    (android, limited_old_sup): 5,
    (web_version, extended_old_sup) : 5,
    (web_version, limited_old_sup) : 2,
    (extended_old_sup, limited_old_sup) : 1/3
}

In [158]:
# creating comparisons
comp_cov_users = ahpy.Compare(cov_users, crit_cov_users, precision=3, random_index='saaty')
comp_impl_speed = ahpy.Compare(impl_speed, crit_impl_speed, precision=3, random_index='saaty')
comp_conv_users = ahpy.Compare(conv_users, crit_conv_users, precision=3, random_index='saaty')
comp_impl_cost = ahpy.Compare(impl_cost, crit_impl_cost, precision=3, random_index='saaty')

comp_criteria = ahpy.Compare('Criteria', criteria_comparisons, precision=3, random_index="saaty")
comp_criteria.add_children([comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost])

In [159]:
comp_criteria.target_weights

{'Publish android app': 0.377,
 'Improve web version for smartphones': 0.301,
 'Limited support for old versions': 0.185,
 'Extended support for old versions': 0.137}

In [160]:
report = comp_criteria.report(show=True)

{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Publish android app": 0.377,
        "Improve web version for smartphones": 0.301,
        "Limited support for old versions": 0.185,
        "Extended support for old versions": 0.137
    },
    "elements": {
        "global_weights": {
            "covered users": 0.417,
            "implementation speed": 0.408,
            "convenience for users": 0.097,
            "implementation cost": 0.078
        },
        "local_weights": {
            "covered users": 0.417,
            "implementation speed": 0.408,
            "convenience for users": 0.097,
            "implementation cost": 0.078
        },
        "consistency_ratio": 0.026
    }
}


In [161]:
pret = prettytable.PrettyTable()

pret.field_names = ["comparison matrix", "consistency ratio (%)"]

def format(cr):
    return "{cr: .1f} %".format(cr=cr*100)

pret.add_row(["criteria", format(cr=comp_criteria.consistency_ratio)])

for i in [comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost]:
    assert(i.consistency_ratio < 0.1)
    pret.add_row([i.name, format(cr=i.consistency_ratio)])
print(pret)

+-----------------------+-----------------------+
|   comparison matrix   | consistency ratio (%) |
+-----------------------+-----------------------+
|        criteria       |          2.6 %        |
|     covered users     |          5.8 %        |
|  implementation speed |          5.2 %        |
| convenience for users |          5.0 %        |
|  implementation cost  |          7.7 %        |
+-----------------------+-----------------------+


In [38]:
import pandas as pd
from fractions import Fraction

In [162]:
! pip install latextable

In [163]:
import latextable
from texttable import Texttable

In [187]:
def latex_matrix(matrix : ahpy.Compare):
    table_1 = Texttable()
    table_1.set_deco(Texttable.VLINES | Texttable.BORDER | Texttable.HLINES)
    table_1.set_cols_align(["c","c","c","c"])
    formatted = [[str(Fraction(g).limit_denominator(10).real) for g in i] for i in matrix._matrix]
    table_1.add_rows(formatted, header=False)
    latex = latextable.draw_latex(table_1, caption=matrix.name)
    index = latex.find(r"\\")
    corrected = latex[:index] + latex[index+2:]
    return corrected

In [185]:
def format(cr):
    return "{cr: .1f} \%".format(cr=cr*100)

crs = []
crs.append(["criteria", format(cr=comp_criteria.consistency_ratio)])
for i in [comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost]:
    assert(i.consistency_ratio < 0.1)
    crs.append([i.name, format(cr=i.consistency_ratio)])
print(crs)


def coefficient_ratio(matrix):
    table_1 = Texttable()
    table_1.set_cols_align(["l","c"])
    table_1.set_deco(Texttable.HEADER | Texttable.VLINES)
    newm = [i for i in matrix]
    newm.insert(0, ["Comparison matrix", "Consistency ratio (\%)"])
    table_1.add_rows(newm, header=True)
    latex = latextable.draw_latex(table_1, caption="Consistency Ratios",)
    return latex


[['criteria', ' 2.6 \\%'], ['covered users', ' 5.8 \\%'], ['implementation speed', ' 5.2 \\%'], ['convenience for users', ' 5.0 \\%'], ['implementation cost', ' 7.7 \\%']]


In [196]:
import os 
import sys 
d = sys.path[0]
print(d)

with open('{}/output.txt'.format(d), 'w') as fp:
    fp.write(
        """\\documentclass{article}
\\usepackage{graphicx} % Required for inserting images
\\usepackage{amsmath} 

\\title{AHP}
\\author{Alexander Gornostaev}
\\date{March 2023}

\\begin{document}

\\maketitle
    """
    )
    for i in [comp_criteria, comp_cov_users, comp_impl_speed, comp_conv_users, comp_impl_cost]:
        fp.write(latex_matrix(i))
        fp.write('\n')
    fp.write(coefficient_ratio(crs))
    fp.write('\n')
    fp.write(r'\end{document}')
    pass


c:\Users\Alexander\Projects\ahp_calc
